In [3]:
pip install openai==0.28



SyntaxError: invalid syntax (3204508438.py, line 1)

In [3]:
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension


SyntaxError: invalid syntax (3280527727.py, line 1)

In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import nest_asyncio
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [2]:
pip install llama-index opik nest_asyncio python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.


d:\projects\llama-4_vs_deepseek-r1\.venv\Scripts\python.exe: No module named pip


In [4]:
import opik
opik.configure(use_local=False)

OPIK: Opik is already configured. You can check the settings by viewing the config file at C:\Users\acer\.opik.config


True

Note: you may need to restart the kernel to use updated packages.


d:\projects\llama-4_vs_deepseek-r1\.venv\Scripts\python.exe: No module named pip


In [5]:
import os
from openai import OpenAI

# Load your OpenAI API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello!"}
    ]
)

print(response.choices[0].message.content)

Hello! How can I assist you today?


# Setup Workflow

In [1]:
import os
from llama_index.llms.groq import Groq


def load_llm(model_option):
    if model_option == "Llama-4":
        llm = Groq(model="meta-llama/llama-4-scout-17b-16e-instruct", api_key=os.getenv("GROQ_API_KEY"))
    else:
        llm = Groq(model="deepseek-r1-distill-llama-70b", api_key=os.getenv("GROQ_API_KEY"))
    return llm

d:\projects\llama-4_vs_deepseek-r1\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
model_name = 'Llama-4'
# model_name = 'DeepSeek-R1'
llm  = load_llm(model_name)

# Trace RAG calls 

In [21]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from opik.integrations.llama_index import LlamaIndexCallbackHandler

# A callback handler tp automatically log all LlamaIndex operations to Opik
opik_callback_handler = LlamaIndexCallbackHandler()

# Integrate handler into LlamaIndex's settings
Settings.callback_manager = CallbackManager([opik_callback_handler])

# Evaluation

In [ ]:
from opik import Opik

client = Opik()
dataset = client.get_or_create_dataset(name="Fake News Detection Dataset")

In [ ]:
import pandas as pd

df = pd.read_csv("crowd_sourced_balanced_dataset.csv")

In [ ]:
# Prepare data for fake news detection evaluation
# Adjust column names based on your dataset structure

text_col = df.columns[0]  # Adjust if needed
label_col = df.columns[1]  # Adjust if needed

fake_news_pairs = [
    {
        "input": row[text_col], 
        "expected_output": "FAKE" if row[label_col] == 1 else "REAL",
        "context": row[text_col]
    } 
    for _, row in df.head(100).iterrows()  # Using first 100 for evaluation
]
fake_news_pairs[0]

Use insert if you're creating the dataset for the first time

In [ ]:
dataset.insert(fake_news_pairs)

In [ ]:
import pickle

# Load the trained fake news detection model
with open('fake_news_model.pkl', 'rb') as f:
    fake_news_model = pickle.load(f)
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

In [ ]:
from opik import track
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    """Preprocess text for fake news detection"""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

@track
def fake_news_classifier(input: str) -> str:
    processed_text = preprocess_text(input)
    text_vectorized = tfidf_vectorizer.transform([processed_text])
    prediction = fake_news_model.predict(text_vectorized)[0]
    return "FAKE" if prediction == 1 else "REAL"

def evaluation_task(x):
    return {
        "output": fake_news_classifier(x['input'])
    }

In [ ]:
from opik.evaluation.metrics import (
    Equals,
    AnswerRelevance
)

# Define the metrics for fake news detection
equals_metric = Equals()  # Exact match for classification
answer_relevance_metric = AnswerRelevance()

In [ ]:
from opik.evaluation import evaluate

evaluation = evaluate(
    dataset=dataset,
    task=evaluation_task,
    experiment_name="Fake News Detection - " + model_name,
    scoring_metrics=[equals_metric, answer_relevance_metric],
    experiment_config={
        "model": "Logistic Regression + TF-IDF"
    }
)